In [2]:
import helpers
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt

# Modeling

In [36]:
import numpy as np
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation
from keras.layers import Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils
np.random.seed(42)

In [26]:
# Каталог с данными для обучения
train_dir = './data/training'

# Каталог с данными для проверки
val_dir = './data/test'

# Каталог с данными для тестирования
test_dir = 'test'

# Размеры изображения
img_width, img_height = 64, 64 #150, 150

# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)

# Количество эпох
epochs = 30

# Размер мини-выборки
batch_size = 16

# Количество изображений для обучения
nb_train_samples = 13220

# Количество изображений для проверки
nb_validation_samples = 505

# Количество изображений для тестирования
#\nb_test_samples = 3750

In [31]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [32]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 13220 images belonging to 9 classes.


In [33]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 255 images belonging to 9 classes.


In [40]:
# создаём модель
model = Sequential()

# первый свёрточный слой
#32 - карты признаков, размер ядра свёртки 3х3,
#вход 3 канальный цвет в-32, ш-32
model.add(Convolution2D(32,(3,3), border_mode='same',
                       input_shape=input_shape, activation='relu'))

# второй свёрточный слой 
#32 - карты признаков, размер ядра свёртки 3х3,
model.add(Convolution2D(32, (3,3), activation='relu'))

# слой подвыборки
# из квадрата 2х2 выбирается максимальное значение
model.add(MaxPooling2D(pool_size=(2, 2)))

# слой регуляризации 
# отключает некоторые нейроны с вероятностью 25% чтобы предотвратить переобучение 
model.add(Dropout(0.25))

#---------------------------------------------
# третий свёрточный слой 
# уже 64 карты признаков 
model.add(Convolution2D(64, (3,3), border_mode='same',
                       activation='relu'))

# четвёртый свёрточный слой 
model.add(Convolution2D(64, (3,3), activation='relu'))

# второй слой подвыборки 
# максимальное значение из квадрата 2x2
model.add(MaxPooling2D(pool_size=(2, 2)))

# слой регуляции
#регуляризация откл нейронов
model.add(Dropout(0.25))
#--------------------------------------------
# Классификатор
# преобразование из двумерного вида в плоский 
model.add(Flatten())
# полносвязный слой
model.add(Dense(512, activation='relu'))
#слой регуляризации
model.add(Dropout(0.5))
#выходкой слой 
model.add(Dense(9, activation='softmax'))

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 64, 3..., activation="relu", padding="same")`
  
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`


In [41]:
# компилируем сеть 
model.compile(loss='categorical_crossentropy', 
              optimizer='SGD',
              metrics=['accuracy'])

In [42]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/30
826/826 [==============================] - 292s 354ms/step - loss: 1.3277 - acc: 0.4763 - val_loss: 2.0040 - val_acc: 0.4510
Epoch 2/30
826/826 [==============================] - 285s 345ms/step - loss: 0.8201 - acc: 0.7157 - val_loss: 1.0539 - val_acc: 0.6157
Epoch 3/30
826/826 [==============================] - 285s 346ms/step - loss: 0.4716 - acc: 0.8441 - val_loss: 0.8606 - val_acc: 0.6471
Epoch 4/30
826/826 [==============================] - 287s 348ms/step - loss: 0.3561 - acc: 0.8851 - val_loss: 0.7562 - val_acc: 0.7412
Epoch 5/30
826/826 [==============================] - 281s 340ms/step - loss: 0.2822 - acc: 0.9044 - val_loss: 0.8249 - val_acc: 0.7647
Epoch 6/30
826/826 [==============================] - 278s 336ms/step - loss: 0.2331 - acc: 0.9206 - val_loss: 0.5400 - val_acc: 0.8000
Epoch 7/30
826/826 [==============================] - 280s 339ms/step - loss: 0.2041 - acc: 0.9310 - val_loss: 0.4970 - val_acc: 0.8392
Epoch 8/30
826/826 [============================

acc: 0.9839, val_acc: 0.9451

In [43]:
model.save_weights('road_sign_model.h5')

In [44]:
# генерируем описание модели в формате json
model_json = model.to_json()

#записываем модель в файл
json_file = open('road_sign_model.json', 'w')
json_file.write(model_json)
json_file.close()

In [45]:
model.save_weights('road_sign_weights.h5')

In [48]:
model.evaluate_generator(train_generator)

[0.02577223068471021, 0.9921331316187595]